In [5]:
!pip install -q transformers datasets evaluate accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 94.3 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
import numpy as np
import pandas as pd
from datasets import Dataset,load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
from transformers import default_data_collator
import torch


In [11]:
df = pd.read_csv('/kaggle/input/squad-v11/SQuAD-v1.2.csv')
ds = Dataset.from_pandas(df).train_test_split(test_size=0.05)


In [12]:
MODEL = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
model = AutoModelForQuestionAnswering.from_pretrained(MODEL)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
max_length = 384
doc_stride = 128

def prepare_features(examples):
    # tokenize with sliding window + offsets
    tokenized = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_map = tokenized.pop("overflow_to_sample_mapping")
    offsets = tokenized.pop("offset_mapping")

    starts, ends = [], []

    # helper to extract (text, start) for the original example index
    def _get_answer(sample_idx):
        # 1) HF style: examples may contain 'answers' (batched -> list of dicts)
        if "answers" in examples and examples["answers"] is not None:
            ans_entry = examples["answers"][sample_idx]
            # ans_entry expected to be a dict with 'text' and 'answer_start' (which may be lists)
            if isinstance(ans_entry, dict):
                texts = ans_entry.get("text", [])
                starts_list = ans_entry.get("answer_start", [])
                text = texts[0] if isinstance(texts, (list, tuple)) and texts else (texts if texts else None)
                start = starts_list[0] if isinstance(starts_list, (list, tuple)) and starts_list else (starts_list if starts_list else None)
                return text, start

        # 2) CSV style: separate columns 'answer' and 'answer_start' (batched -> lists)
        ans_col = examples.get("answer")
        ans_start_col = examples.get("answer_start")
        if ans_col is not None and ans_start_col is not None:
            return ans_col[sample_idx], ans_start_col[sample_idx]

        # 3) not found
        return None, None

    for i, offset in enumerate(offsets):
        sample_idx = sample_map[i]
        answer_text, answer_start = _get_answer(sample_idx)

        if answer_text is None or answer_start is None:
            starts.append(0); ends.append(0); continue

        answer_start = int(answer_start)
        answer_end = answer_start + len(answer_text)

        seq_ids = tokenized.sequence_ids(i)
        # find context token span
        context_start = next(j for j, s in enumerate(seq_ids) if s == 1)
        context_end = len(seq_ids) - 1 - next(j for j, s in enumerate(reversed(seq_ids)) if s == 1)

        # if answer not fully inside this window -> CLS token (0)
        if not (answer_start >= offset[context_start][0] and answer_end <= offset[context_end][1]):
            starts.append(0); ends.append(0)
        else:
            ts = context_start
            while ts <= context_end and offset[ts][0] <= answer_start:
                ts += 1
            te = context_end
            while te >= context_start and offset[te][1] >= answer_end:
                te -= 1
            starts.append(ts - 1)
            ends.append(te + 1)

    tokenized["start_positions"] = starts
    tokenized["end_positions"] = ends
    return tokenized


In [18]:
train = ds["train"].select(range(4000))   
valid = ds["test"].select(range(500))
tokenized_train = train.map(prepare_features, batched=True, remove_columns=train.column_names)
tokenized_valid = valid.map(prepare_features, batched=True, remove_columns=valid.column_names)


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="qa-run",
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    logging_steps=200,
    save_strategy="no",
    fp16=torch.cuda.is_available(),
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipykernel_36/127316169.py:11: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


<IPython.core.display.Javascript object>